In [1]:
import torch

/home/shuchengdu/miniconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


获取硬件条件

MPS(Metal Performance Shaders)

In [2]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device")

using cuda device


# Load Dataset

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Get FashionMNIST

Train and Test

In [5]:
train_dataset = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_dataset = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Create Model

- 前向传播
- 转换数据格式（物理）

网络 + 数据 都是在设备之上

In [6]:
from torch import nn

In [7]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


# Train and Test

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0
  with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      pred = model(X)
      loss = loss_fn(pred, y)
      test_loss += loss.item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

准确度就是58.6上不去了

In [16]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss, optimizer)
  test(test_dataloader, model, loss)
print("Done!")

Epoch 1
-------------------------------
loss: 1.612472 [    0/60000]
loss: 1.565002 [ 6400/60000]
loss: 1.494106 [12800/60000]
loss: 1.471718 [19200/60000]
loss: 1.219405 [25600/60000]
loss: 1.438121 [32000/60000]
loss: 1.395477 [38400/60000]
loss: 1.421327 [44800/60000]
loss: 1.576724 [51200/60000]
loss: 1.267216 [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.387403 

Epoch 2
-------------------------------
loss: 1.612472 [    0/60000]
loss: 1.565002 [ 6400/60000]
loss: 1.494106 [12800/60000]
loss: 1.471718 [19200/60000]
loss: 1.219405 [25600/60000]
loss: 1.438121 [32000/60000]
loss: 1.395477 [38400/60000]
loss: 1.421327 [44800/60000]
loss: 1.576724 [51200/60000]
loss: 1.267216 [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.387403 

Epoch 3
-------------------------------
loss: 1.612472 [    0/60000]
loss: 1.565002 [ 6400/60000]
loss: 1.494106 [12800/60000]
loss: 1.471718 [19200/60000]
loss: 1.219405 [25600/60000]
loss: 1.438121 [32000/60000]
loss: 1.395477 [38400/

# Saving and Load

In [20]:
from pathlib import Path
model_dic = Path("model")
if not model_dic.is_dir():
    model_dic.mkdir()

torch.save(model.state_dict(), model_dic / "get_started.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [1]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_dic / "get_started.pth"))

NameError: name 'NeuralNetwork' is not defined

# Inference

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_dataset[0][0], test_dataset[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
